<a href="https://colab.research.google.com/github/augustxj/face_recognition_YOLOv8/blob/main/face_recognition_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Make sure you're running in TPU enviroment!!!!!!!!!


In [ ]:
!pip install ultralytics face-recognition


In [ ]:
import face_recognition
import numpy as np
import cv2
import os

from google.colab.patches import cv2_imshow
from ultralytics import YOLO

In [ ]:
# setting YOLO model
model = YOLO('yolov8n.pt')

###Handling the images

In [ ]:
'''Make sure the name of the images ARE THE LABELS OF IT. Ex: Elvis.png'''
#creating the base folder
data = 'labels' #naming the folder
os.makedirs(data, exist_ok=True) #creating the folder

#get the images
!wget -O {data}/sherlock.jpg https://ichef.bbci.co.uk/images/ic/1920x1080/p01pw5gv.jpg
!wget -O {data}/watson.png https://static.wikia.nocookie.net/p__/images/2/21/John_Watson-_Martin_Freeman.png/revision/latest?cb=20150522195709&path-prefix=protagonist
print("Images saved in:", data)

'''if you wish to use images from your own device, delete ONLY the !wget lines above and uncomment the lines below:
'''
#from google.colab import files
#import shutil
#uploaded = files.upload()
#for filename in uploaded.keys():
    #shutil.move(filename, os.path.join(data, filename))
#print("Images saved in:", data)

known_face_encodings = []
known_face_names = []

In [ ]:
# Load know images
def detect_know_faces(directory):
    # Iterate through all images in the folder
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
            image_path = os.path.join(directory, filename)
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                known_face_encodings.append(face_encodings[0])

                # using the file name as the person's name
                name = os.path.splitext(filename)[0]
                known_face_names.append(name)

    print(f"{len(data)} images loaded. {len(known_face_encodings)} faces recognized.")

def recognize_faces(image_path):
    for filename in os.listdir(image_path):
        file_path = os.path.join(image_path, filename)  # Full image path

        # Verify if it's an image
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.webp')):
            continue

        # Load the image
        image_to_recognize = face_recognition.load_image_file(file_path)
        rgb_image = cv2.cvtColor(image_to_recognize, cv2.COLOR_BGR2RGB)

        # Detect faces
        face_locations = face_recognition.face_locations(rgb_image)
        face_encodings_to_check = face_recognition.face_encodings(rgb_image, face_locations)

        # Error Handling
        if not known_face_encodings:
            raise ValueError(f"No faces were found in the folder {image_path}. "
                             "Make sure the reference images contain visible faces.")

        recognized_names = []

        # Process image
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings_to_check):
            # Verify matches
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if any(matches):  # Check if there are any matches
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

            recognized_names.append(name)

            # Draw rectangle and label
            cv2.rectangle(rgb_image, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(rgb_image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Reduce the image size for display
        scale_percent = 50  # Percentage of the original size
        width = int(rgb_image.shape[1] * scale_percent / 100)
        height = int(rgb_image.shape[0] * scale_percent / 100)
        dim = (width, height)

        resized_image = cv2.resize(rgb_image, dim, interpolation=cv2.INTER_AREA)

        # Print recognized names
        unique_names = set(recognized_names)  # Remove duplicates
        if "Unknown" in unique_names and len(unique_names) == 1:
            print(f"No recognized faces in {filename}.")
        else:
            print(f"Recognized persons in {filename}: {', '.join(unique_names)}")

        # Display the image
        cv2_imshow(resized_image)

In [ ]:
detect_know_faces(data)

In [ ]:
# Creating the test folder
test = 'test images'
os.makedirs(test, exist_ok=True)

#loading the images
!wget -P {test} https://s.yimg.com/ny/api/res/1.2/qCHpM43J7GVlXLEloCX1wg--/YXBwaWQ9aGlnaGxhbmRlcjt3PTI0MDA7aD0xNjAwO2NmPXdlYnA-/https://s.yimg.com/os/creatr-uploaded-images/2023-01/dd141390-8b6d-11ed-9b7a-5a9a507f9acb
!wget -p {test} https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEgI7C_9IXD7QNw13r6TxWmOamxduxSKComAyBlZEAEPBBPhSseipvemd0HALoCVrDLTlR-3NNKG4OInDVAnPsIyj-U6MuKZtQep5N5WA74e05fwaq5Ci5IRN1VinplcSlZ2cDZfIgFKFDY/s1600/cast+and+crew+BBC+SH+1.jpeg

# Check if images exist before proceeding
if any(fname.lower().endswith(('.png', '.jpg', '.jpeg')) for fname in os.listdir(test)):
    recognize_faces(test)
else:
    print(f"No images found in {test}. Check the URLs and try again.")